# Machine Learning

I will now move into the final phase of the project and run a few different recommendation engines. Before that though, there is a bit of preprocessing we need to take care of. I will walk through those steps here.

In [332]:
#Import packages
import pandas as pd
import numpy as np
import time
import turicreate as tc
from sklearn.cross_validation import train_test_split

In [333]:
#Load data
df = pd.read_csv('Data/train_ver2_CLEAN')

In [334]:
df.head()

,index,date,customer_code,employee_index,customer_country,sex,age,first_contract_date,new_customer_index,customer_seniority,...,deceased_index,province_name,activity_index,household_gross_income,segmentation,month,month_id,month_next_id,variable,value
0,265696,2015-03-28,118972.0,N,ES,V,44.0,2000-05-23,0.0,182.0,...,N,MADRID,1.0,172178.94,01 - TOP,3.0,3,4,gurantees,Added
1,2484859,2015-02-28,116132.0,N,ES,V,45.0,1998-12-22,0.0,199.0,...,N,VALENCIA,1.0,92877.72,02 - PARTICULARES,2.0,2,3,current_accounts,Added
2,2998969,2015-02-28,1340674.0,N,ES,H,23.0,2014-10-24,0.0,9.0,...,N,MADRID,1.0,259009.38,03 - UNIVERSITARIO,2.0,2,3,current_accounts,Dropped
3,2392444,2015-02-28,190613.0,N,ES,H,38.0,2000-11-29,0.0,175.0,...,N,MADRID,1.0,68797.14,02 - PARTICULARES,2.0,2,3,current_accounts,Dropped
4,2377004,2015-02-28,171133.0,N,ES,H,43.0,2003-05-22,0.0,146.0,...,N,MADRID,1.0,144579.54,02 - PARTICULARES,2.0,2,3,current_accounts,Dropped


### Data Preprocessing

In [335]:
#I'm noticing that the 'customer_code' values are in float form. I need to convert these to integers.
df.customer_code = df.customer_code.astype('int')

In [363]:
#Convert 'Added'/'Dropped' to 1 and 0
df.value = df.value.replace('Added', 1)
df.value = df.value.replace('Dropped', 0)

Since we do not have a product code or product ID for the different services the bank offers, I want to begin by converting them to unique IDs. This will help process them into a more helpful form for the recommendation engines down the road.

In [336]:
#Check out unique values for products
df.variable.unique()

array(['gurantees', 'current_accounts', 'derivada_account',
       'payroll_account', 'junior_account', 'mas_particular_account',
       'particular_account', 'particular_plus_account',
       'shortterm_deposits', 'mediumterm_deposits', 'longterm_deposits',
       'online_account', 'funds', 'mortgage', 'pensions', 'loans',
       'taxes', 'credit_card', 'securities', 'home_account', 'payroll',
       'pensions_2', 'direct_debit'], dtype=object)

In [337]:
#Create a new column, 'product_code' that consists of all of the services the bank offers
df['product_code'] = df['variable']

In [340]:
#I will now replace each unique product with a designated 'code'
df.product_code = df.product_code.replace('gurantees', 1)
df.product_code = df.product_code.replace('current_accounts', 2)
df.product_code = df.product_code.replace('derivada_account', 3)
df.product_code = df.product_code.replace('payroll_account', 4)
df.product_code = df.product_code.replace('junior_account', 5)
df.product_code = df.product_code.replace('mas_particular_account', 6)
df.product_code = df.product_code.replace('particular_account', 7)
df.product_code = df.product_code.replace('particular_plus_account', 8)
df.product_code = df.product_code.replace('shortterm_deposits', 9)
df.product_code = df.product_code.replace('mediumterm_deposits', 10)
df.product_code = df.product_code.replace('longterm_deposits', 11)
df.product_code = df.product_code.replace('online_account', 12)
df.product_code = df.product_code.replace('funds', 13)
df.product_code = df.product_code.replace('mortgage', 14)
df.product_code = df.product_code.replace('pensions', 15)
df.product_code = df.product_code.replace('loans', 16)
df.product_code = df.product_code.replace('taxes', 17)
df.product_code = df.product_code.replace('credit_card', 18)
df.product_code = df.product_code.replace('securities', 19)
df.product_code = df.product_code.replace('home_account', 20)
df.product_code = df.product_code.replace('payroll', 21)
df.product_code = df.product_code.replace('pensions_2', 22)
df.product_code = df.product_code.replace('direct_debit', 23)

In [341]:
#Confirm that it worked
df.head()

,index,date,customer_code,employee_index,customer_country,sex,age,first_contract_date,new_customer_index,customer_seniority,...,province_name,activity_index,household_gross_income,segmentation,month,month_id,month_next_id,variable,value,product_code
0,265696,2015-03-28,118972,N,ES,V,44.0,2000-05-23,0.0,182.0,...,MADRID,1.0,172178.94,01 - TOP,3.0,3,4,gurantees,1,1
1,2484859,2015-02-28,116132,N,ES,V,45.0,1998-12-22,0.0,199.0,...,VALENCIA,1.0,92877.72,02 - PARTICULARES,2.0,2,3,current_accounts,1,2
2,2998969,2015-02-28,1340674,N,ES,H,23.0,2014-10-24,0.0,9.0,...,MADRID,1.0,259009.38,03 - UNIVERSITARIO,2.0,2,3,current_accounts,0,2
3,2392444,2015-02-28,190613,N,ES,H,38.0,2000-11-29,0.0,175.0,...,MADRID,1.0,68797.14,02 - PARTICULARES,2.0,2,3,current_accounts,0,2
4,2377004,2015-02-28,171133,N,ES,H,43.0,2003-05-22,0.0,146.0,...,MADRID,1.0,144579.54,02 - PARTICULARES,2.0,2,3,current_accounts,0,2


Now that we have the data in a form that works for the recommendation engines. I need to create a new dataframe. The goal is to group the original dataframe by 'customer_code', figure out if the customer purchased a specific item, and list the 'product_code' along with it. I will then take the sum of each purchase to create a new column, 'purchase_counts'. This table will be used as an input for the model later on.

In [342]:
#Perform groupby operation
df2 = df.groupby('customer_code')['value'].agg(np.sum).reset_index(name='purchase_counts')
df2['product_code'] = df['product_code']

In [343]:
#Confirm that it worked
df2.head()

,customer_code,purchase_counts,product_code
0,15900,1,1
1,15906,1,2
2,15911,1,2
3,15923,1,2
4,15927,1,2


I will now create a dummy for marking whether a customer bought a specific item or not. This will also be useful for modeling later.

In [344]:
#Define create_data_dummy function
def create_data_dummy(data):
    data_dummy = data.copy()
    data_dummy['purchase_dummy'] = 1
    return data_dummy
data_dummy = create_data_dummy(df2)

We now want to normalize item values across users. We will start by normalizing purchase frequency for each item across users by creating a user-item matrix

In [345]:
#Create user-item matrix
df_matrix = pd.pivot_table(df2, values='purchase_counts', index='customer_code', columns='product_code')
df_matrix.head()

product_code,1,2,3,4,5,6,7,8,9,10,...,12,13,14,15,16,17,18,19,20,21
customer_code,,,,,,,,,,,,,,,,,,,,,
15900,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15906,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15911,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15923,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15927,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [346]:
df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
df_matrix_norm.head()

product_code,1,2,3,4,5,6,7,8,9,10,...,12,13,14,15,16,17,18,19,20,21
customer_code,,,,,,,,,,,,,,,,,,,,,
15900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15906,NaN,0.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15911,NaN,0.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15923,NaN,0.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15927,NaN,0.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [347]:
d = df_matrix_norm.reset_index() 
d.index.names = ['scaled_purchase_freq'] 
data_norm = pd.melt(d, id_vars=['customer_code'], value_name='scaled_purchase_freq').dropna()

print(data_norm.shape)
data_norm.head()

(24572, 3)


,customer_code,product_code,scaled_purchase_freq
24574,15906,2,0.2
24575,15911,2,0.2
24576,15923,2,0.2
24577,15927,2,0.2
24578,15937,2,0.0


### Modeling

We have now normalized the purchase history for each customer on a scale of 0 o 1. A 0 represents no purchases and a 1 representing the most purchases for a customer. 

Now that we have prepared the data for modeling, we will go ahead and perform a train test split. Obviously, the training data will be used for performing the model and the testing data will be used to evaluate the model.

In [348]:
#Define split function
def split_data(data):
    train, test = train_test_split(data, test_size = .2)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data

We have 3 different data sets, one for purchase counts, puchase dummy and scaled purchase counts. We will go ahead and split these for modeling.

In [349]:
#Split each data set
train_data, test_data = split_data(df2)
train_data_dummy, test_data_dummy = split_data(data_dummy)
train_data_norm, test_data_norm = split_data(data_norm)

The ultimate goal is for us to be able to recommend products for specific users based on what similiar users previously purchased. We will be doing this with the collaborative filtering method. However, I would like to begin by performing a 'popularity model' that will simply recommend the most popular products for each user. I will do this as a baseline so that we can evaluate our collaborative filtering method against it. 

In [350]:
#Define objects for model
user_id = 'customer_code'
item_id = 'product_code'
users_to_recommend = list(df['customer_code'])

#Number of items to recommend
n_rec = 15 
#Display the first few rows in an output dataset
n_display = 30 

In [351]:
#Here I am defining a function, model, that will input data, model name, user id, product id
#target, users to recommend, number of items to recommend and number of rows to display.
#This is for ease of calling each model type. 

def model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display):
    if name == 'popularity':
        model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)
    elif name == 'cosine':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='cosine')
    elif name == 'pearson':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='pearson')
        
    recom = model.recommend(users=users_to_recommend, k=n_rec)
    recom.print_rows(n_display)
    return model

### Popularity Models

In [352]:
#Popularity model using purchase counts
name = 'popularity'
target = 'purchase_counts'
popularity = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 19658 observations with 19658 users and 21 items.

Data prepared in: 0.086004s

19658 observations to process; with 21 unique items.

recommendations finished on 1000/39964 queries. users per second: 62515.6

...

recommendations finished on 36000/39964 queries. users per second: 174926

recommendations finished on 37000/39964 queries. users per second: 175768

recommendations finished on 38000/39964 queries. users per second: 168238

recommendations finished on 39000/39964 queries. users per second: 163988

+---------------+--------------+--------------------+------+
| customer_code | product_code |       score        | rank |
+---------------+--------------+--------------------+------+
|     118972    |      21      | 1.0839064649243466 |  1   |
|     118972    |      1       |        1.0         |  2   |
|     118972    |      19      | 0.9059561128526645 |  3   |
|     118972    |      20      |        0.9         |  4   |
|     118972    |      16      | 0.8888888888888888 |  5   |
|     118972    |      18      | 0.8704833411543876 |  6   |
|     118972    |      17      | 0.8604651162790697 |  7   |
|     118972    |      6       |     0.8140625      |  8   |
|     118972    |      13      | 0.7820224719101123 |  9   |
|     118972    |      12      | 0.7814465408805031 |  10  |
|     118972    |      7       | 0.7802874743326489 |  11  |
|     118972    |      4       | 0.7781672508763144 |  12  |
|     118972    |      3       | 0.7692307692307693 |  13  |
|     118972    |      1

In [353]:
#Popularity model using purchase dummies
name = 'popularity'
target = 'purchase_dummy'
pop_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_counts;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 19658 observations with 19658 users and 21 items.

Data prepared in: 0.048508s

...

recommendations finished on 36000/39964 queries. users per second: 211428

recommendations finished on 37000/39964 queries. users per second: 213541

recommendations finished on 38000/39964 queries. users per second: 213028

recommendations finished on 39000/39964 queries. users per second: 187534

+---------------+--------------+-------+------+
| customer_code | product_code | score | rank |
+---------------+--------------+-------+------+
|     118972    |      4       |  1.0  |  1   |
|     118972    |      18      |  1.0  |  2   |
|     118972    |      21      |  1.0  |  3   |
|     118972    |      11      |  1.0  |  4   |
|     118972    |      13      |  1.0  |  5   |
|     118972    |      9       |  1.0  |  6   |
|     118972    |      12      |  1.0  |  7   |
|     118972    |      6       |  1.0  |  8   |
|     118972    |      10      |  1.0  |  9   |
|     118972    |      19      |  1.0  |  10  |
|     118972    |      7       |  1.0  |  11  |
|     118972    |      3       |  1.0  |  12  |
|     118972    |      17      |  1.0  |  13  |
|     118972    |      8       |  1.0  |  14  |
|     118972    |      14      |  1.0  |  15  |
|     116132    |      4       |  1.0  |  1   |
|     116132    |      18      |  1.0  |  2   |
|     116132    |      21      |  1.0  |

In [354]:
#Popularity model using scaled purchase frequencies
name = 'popularity'
target = 'scaled_purchase_freq'
pop_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 19657 observations with 19657 users and 20 items.

Data prepared in: 0.050549s

19657 observations to process; with 20 unique items.

recommendations finished on 1000/39964 queries. users per second: 92910.9

...

recommendations finished on 36000/39964 queries. users per second: 223225

recommendations finished on 37000/39964 queries. users per second: 220539

recommendations finished on 38000/39964 queries. users per second: 209518

recommendations finished on 39000/39964 queries. users per second: 206858

+---------------+--------------+---------------------+------+
| customer_code | product_code |        score        | rank |
+---------------+--------------+---------------------+------+
|     118972    |      20      |         0.4         |  1   |
|     118972    |      3       | 0.35714285714285715 |  2   |
|     118972    |      16      | 0.26666666666666666 |  3   |
|     118972    |      15      |  0.2391304347826087 |  4   |
|     118972    |      19      | 0.23166666666666666 |  5   |
|     118972    |      5       | 0.22988505747126436 |  6   |
|     118972    |      10      | 0.22510822510822517 |  7   |
|     118972    |      17      | 0.20821114369501467 |  8   |
|     118972    |      6       | 0.20219435736677116 |  9   |
|     118972    |      14      | 0.18055555555555555 |  10  |
|     118972    |      8       | 0.17647058823529413 |  11  |
|     118972    |      9       | 0.16319444444444445 |  12  |
|     118972    |      7       |  0.1621621621621628 |  13  |
|     11

In the models above, we have a list of users, the most popular items, their score and their rank. This is showing us what each user was recommended. You'll notice that each user is recommended the same list of items in the exact same order. This is exactly what we want. We are looking for the most popular, and the most popular items should not change depending on the user. The different popularity models are taking in the 3 different data sets. The popularity of the product changes depending on the data we input.

### Collaborative Filtering Models

As mentioned above, the ultimate goal is to be able to identify users who are similar and recommend products to them based on what other users like them bought that they have not already purchased themselves. The question is: How do we identify which users are similar?

We will use a few different methods for defining similarity:

    1. Create a user-item matrix where indexes represent customer IDs and column values represent product IDs.


    2. Create an item-to-item similarity matrix. We will basically be calculating how 'similar' one product is to another product. There are a few different ways to do this, but for our example we will focus on the cosine and pearson measures of similarity.
    
        -For calculating the similarity between two products, we will look at all of the customers who purchased both of these items
    
        -Then we will create two-item vectors and find the cosine or pearson angle/distance between the vectors. Total similarity is equivalent to a cosine value of 1 and an angle of 90 degrees represents a cosine of 0, or no similarity.
    

    3. We will then predict the likelihood of a customer to buy a product for products they have not already purchased.

### Cosine Similarity Models

In [355]:
#Cosine similarity model using purchase counts
name = 'cosine'
target = 'purchase_counts'
cos = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 19658 observations with 19658 users and 21 items.

Data prepared in: 0.048829s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 36000/39964 queries. users per second: 71693.2

recommendations finished on 37000/39964 queries. users per second: 71134.8

recommendations finished on 38000/39964 queries. users per second: 72431.6

recommendations finished on 39000/39964 queries. users per second: 63518.4

+---------------+--------------+---------------------+------+
| customer_code | product_code |        score        | rank |
+---------------+--------------+---------------------+------+
|     118972    |      21      |  0.4760633276694447 |  1   |
|     118972    |      1       |  0.3921568627450981 |  2   |
|     118972    |      19      |  0.2981129755977626 |  3   |
|     118972    |      20      |  0.2921568627450981 |  4   |
|     118972    |      16      | 0.28104575163398693 |  5   |
|     118972    |      18      |  0.2626402038994857 |  6   |
|     118972    |      17      | 0.25262197902416783 |  7   |
|     118972    |      6       |  0.2062193627450981 |  8   |
|     118972    |      13      |  0.1741793346552104 |  9   |
|     118972    |      12      | 0.17360340362560123 |  10  |
|     118972    |      7       | 0.17244433707774698 |  11  |
|     118972    |      4       | 0.17032411362141253 |  12  |
|     118972    |      3       | 0.16138763197586736 |  13  |
|     11

In [356]:
#Cosine similarity model using purchase dummies
name = 'cosine'
target = 'purchase_dummy'
cos_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_counts;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 19658 observations with 19658 users and 21 items.

Data prepared in: 0.047318s

...

recommendations finished on 36000/39964 queries. users per second: 52782.4

recommendations finished on 37000/39964 queries. users per second: 53546.6

recommendations finished on 38000/39964 queries. users per second: 54030.9

recommendations finished on 39000/39964 queries. users per second: 54125.1

+---------------+--------------+-------+------+
| customer_code | product_code | score | rank |
+---------------+--------------+-------+------+
|     118972    |      4       |  0.0  |  1   |
|     118972    |      18      |  0.0  |  2   |
|     118972    |      21      |  0.0  |  3   |
|     118972    |      11      |  0.0  |  4   |
|     118972    |      13      |  0.0  |  5   |
|     118972    |      9       |  0.0  |  6   |
|     118972    |      12      |  0.0  |  7   |
|     118972    |      6       |  0.0  |  8   |
|     118972    |      10      |  0.0  |  9   |
|     118972    |      19      |  0.0  |  10  |
|     118972    |      7       |  0.0  |  11  |
|     118972    |      3       |  0.0  |  12  |
|     118972    |      17      |  0.0  |  13  |
|     118972    |      8       |  0.0  |  14  |
|     118972    |      14      |  0.0  |  15  |
|     116132    |      4       |  0.0  |  1   |
|     116132    |      18      |  0.0  |  2   |
|     116132    |      21      |  0.0  |

In [357]:
#Cosine similarity model using scaled purchase frequencies
name = 'cosine' 
target = 'scaled_purchase_freq' 
cos_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 19657 observations with 19657 users and 20 items.

Data prepared in: 0.052753s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 36000/39964 queries. users per second: 70076.4

recommendations finished on 37000/39964 queries. users per second: 69089.5

recommendations finished on 38000/39964 queries. users per second: 69827

recommendations finished on 39000/39964 queries. users per second: 69952

+---------------+--------------+----------------------+------+
| customer_code | product_code |        score         | rank |
+---------------+--------------+----------------------+------+
|     118972    |      20      | 0.27182361733931265  |  1   |
|     118972    |      3       | 0.22896647448216978  |  2   |
|     118972    |      16      |  0.1384902840059793  |  3   |
|     118972    |      15      | 0.11095405212192133  |  4   |
|     118972    |      19      | 0.10349028400597929  |  5   |
|     118972    |      5       | 0.10170867481057697  |  6   |
|     118972    |      10      | 0.09693184244753772  |  7   |
|     118972    |      17      |  0.0800347610343273  |  8   |
|     118972    |      6       | 0.07401797470608379  |  9   |
|     118972    |      14      | 0.052379172894868126 |  10  |
|     118972    |      8       | 0.04829420557460676  |  11  |
|     118972    |      9       | 0.035018061783757076 |  12  |
|     118972    |      7       | 0.03398577950147544  |

### Pearson simliarity models

In [358]:
#Pearson similarity model using purchase counts
name = 'pearson'
target = 'purchase_counts'
pear = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 19658 observations with 19658 users and 21 items.

Data prepared in: 0.045586s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 36000/39964 queries. users per second: 62228.2

recommendations finished on 37000/39964 queries. users per second: 62746.9

recommendations finished on 38000/39964 queries. users per second: 63096.9

recommendations finished on 39000/39964 queries. users per second: 63711.5

+---------------+--------------+---------------------+------+
| customer_code | product_code |        score        | rank |
+---------------+--------------+---------------------+------+
|     118972    |      21      |  0.4760633276694447 |  1   |
|     118972    |      1       |  0.3921568627450981 |  2   |
|     118972    |      19      |  0.2981129755977626 |  3   |
|     118972    |      20      |  0.2921568627450981 |  4   |
|     118972    |      16      | 0.28104575163398693 |  5   |
|     118972    |      18      |  0.2626402038994857 |  6   |
|     118972    |      17      | 0.25262197902416783 |  7   |
|     118972    |      6       |  0.2062193627450981 |  8   |
|     118972    |      13      |  0.1741793346552104 |  9   |
|     118972    |      12      | 0.17360340362560123 |  10  |
|     118972    |      7       | 0.17244433707774698 |  11  |
|     118972    |      4       | 0.17032411362141253 |  12  |
|     118972    |      3       | 0.16138763197586736 |  13  |
|     11

In [359]:
#Pearson similarity model using purchase dummies
name = 'pearson'
target = 'purchase_dummy'
pear_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_counts;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 19658 observations with 19658 users and 21 items.

Data prepared in: 0.047726s

...

recommendations finished on 36000/39964 queries. users per second: 77214.6

recommendations finished on 37000/39964 queries. users per second: 76985.2

recommendations finished on 38000/39964 queries. users per second: 78070.1

recommendations finished on 39000/39964 queries. users per second: 77271

+---------------+--------------+-------+------+
| customer_code | product_code | score | rank |
+---------------+--------------+-------+------+
|     118972    |      4       |  0.0  |  1   |
|     118972    |      18      |  0.0  |  2   |
|     118972    |      21      |  0.0  |  3   |
|     118972    |      11      |  0.0  |  4   |
|     118972    |      13      |  0.0  |  5   |
|     118972    |      9       |  0.0  |  6   |
|     118972    |      12      |  0.0  |  7   |
|     118972    |      6       |  0.0  |  8   |
|     118972    |      10      |  0.0  |  9   |
|     118972    |      19      |  0.0  |  10  |
|     118972    |      7       |  0.0  |  11  |
|     118972    |      3       |  0.0  |  12  |
|     118972    |      17      |  0.0  |  13  |
|     118972    |      8       |  0.0  |  14  |
|     118972    |      14      |  0.0  |  15  |
|     116132    |      4       |  0.0  |  1   |
|     116132    |      18      |  0.0  |  2   |
|     116132    |      21      |  0.0  |

In [360]:
#Pearson similarity model using scaled purchase frequencies
name = 'pearson'
target = 'scaled_purchase_freq'
pear_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 19657 observations with 19657 users and 20 items.

Data prepared in: 0.047062s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 36000/39964 queries. users per second: 60104.8

recommendations finished on 37000/39964 queries. users per second: 60548.6

recommendations finished on 38000/39964 queries. users per second: 61466.7

recommendations finished on 39000/39964 queries. users per second: 62443.7

+---------------+--------------+----------------------+------+
| customer_code | product_code |        score         | rank |
+---------------+--------------+----------------------+------+
|     118972    |      20      |  0.2718236173393127  |  1   |
|     118972    |      3       |  0.2289664744821698  |  2   |
|     118972    |      16      | 0.13849028400597932  |  3   |
|     118972    |      15      | 0.11095405212192144  |  4   |
|     118972    |      19      | 0.10349028400597932  |  5   |
|     118972    |      5       | 0.10170867481057702  |  6   |
|     118972    |      10      |  0.0969318424475378  |  7   |
|     118972    |      17      | 0.08003476103432733  |  8   |
|     118972    |      6       | 0.07401797470608382  |  9   |
|     118972    |      14      | 0.05237917289486815  |  10  |
|     118972    |      8       | 0.048294205574606786 |  11  |
|     118972    |      9       | 0.035018061783757104 |  12  |
|     118972    |      7       | 0.033985779501475466 |

### Model Evaluation

It is now time to evaluate the models we have performed. We will be using three different evaluation methods: 

    1. Root Mean Squared Error
        - The smaller the RMSE value, the better our model performed
        
    2. Precision
        - What percentage of products that a user buys that are actually recommended?
        
    3. Recall
        - How many of the recommended items did the user actually like?

In [361]:
#For simplicity, create variables to use in calling the evaluation methods
models_w_counts = [popularity, cos, pear]
models_w_dummy = [pop_dummy, cos_dummy, pear_dummy]
models_w_norm = [pop_norm, cos_norm, pear_norm]

names_w_counts = ['Popularity Model on Purchase Counts', 'Cosine Similarity on Purchase Counts', 'Pearson Similarity on Purchase Counts']
names_w_dummy = ['Popularity Model on Purchase Dummy', 'Cosine Similarity on Purchase Dummy', 'Pearson Similarity on Purchase Dummy']
names_w_norm = ['Popularity Model on Scaled Purchase Counts', 'Cosine Similarity on Scaled Purchase Counts', 'Pearson Similarity on Scaled Purchase Counts']

In [362]:
eval_counts = tc.recommender.util.compare_models(test_data, models_w_counts, model_names=names_w_counts)
eval_dummy = tc.recommender.util.compare_models(test_data_dummy, models_w_dummy, model_names=names_w_dummy)
eval_norm = tc.recommender.util.compare_models(test_data_norm, models_w_norm, model_names=names_w_norm)

PROGRESS: Evaluate model Popularity Model on Purchase Counts


recommendations finished on 1000/4915 queries. users per second: 276396

recommendations finished on 2000/4915 queries. users per second: 267237

recommendations finished on 3000/4915 queries. users per second: 258911

recommendations finished on 4000/4915 queries. users per second: 133936

...

recommendations finished on 1000/4915 queries. users per second: 307882

recommendations finished on 2000/4915 queries. users per second: 125857

recommendations finished on 3000/4915 queries. users per second: 89304.3

recommendations finished on 4000/4915 queries. users per second: 90569.5


Precision and recall summary statistics by cutoff
+--------+-----------------------+-----------------------+
| cutoff |     mean_precision    |      mean_recall      |
+--------+-----------------------+-----------------------+
|   1    | 0.0008138351983723276 | 0.0008138351983723276 |
|   2    | 0.0008138351983723274 | 0.0016276703967446548 |
|   3    | 0.0006781959986436083 |  0.002034587995930824 |
|   4    | 0.0009155645981688709 | 0.0036622583926754835 |
|   5    |  0.004231943031536124 |  0.021159715157680618 |
|   6    | 0.0037639877924720345 |  0.02258392675483218  |
|   7    | 0.0038075861066705365 |   0.0266531027466938  |
|   8    |  0.007299084435401806 |  0.058392675483214446 |
|   9    |  0.010014694246637246 |  0.09013224821973527  |
|   10   |  0.00929806714140388  |  0.09298067141403854  |
+--------+-----------------------+-----------------------+
[10 rows x 3 columns]


Overall RMSE: 0.20344418438518055

Per User RMSE (best)
+---------------+------+-------+
| customer

### Evaluation Summary

Results based on RMSE:

1. Popularity on purchase counts: 0.8286392012124091
2. Cosine similarity on purchase counts: 1.0191326912034409
3. Pearson similarity on purchase counts: 1.0191326912034409


4. Popularity model on purchase dummy: 0.0
5. Cosine similarity on purchase dummy: 1.0
6. Pearson similarity on purchase dummy: 1.0


7. Popularity model on scaled purchase counts: 0.1558247832874474
8. Cosine similarity on scaled purchase counts: 0.2034441843851805
9. Pearson similarity on scaled purchase counts: 0.20344418438518055

As we can see, the RMSE score shows that the scaled purchase count models are the strongest. Specifically, the popularity model. However, we are going to throw out the popularity model, because remember - our goal is to utilize collaborative filtering. Therefore, it looks like the Cosine similarity and Pearson similiarity models on scaled purchase counts are the strongest as they relate to the RMSE evaluation method.

When it comes to precision and recall, the goal is to be as close to 1 as possible. This is best seen in the cosine and pearson similiarity models using the purchase dummy data.

### Final Evaluation:

Based on the evaluation metrics, I have chosen to go with either the pearson similarity model or cosine similarity model for purchase dummy as my final model (They both have nearly the same scores for precision and recall, so we can't go wrong with either one). The RMSE is not as strong for those, but the precision and recall values are much better than the other models.